In [4]:
import numpy as np
a=3   #Latice constant in Åmstrongs
sigma=1 #Very arbitrary value
Kmax=6  
maxK=1
NrOfK=maxK*2+1
Len_k=3
U=100
A=-U*np.sqrt(2 *np.pi)*sigma/a

In [10]:
Energies=np.zeros(shape=(NrOfK, NrOfK))
kVect=np.zeros((2,Len_k*2+1,Len_k*2+1))

for kx in range(-Len_k,Len_k+1):    
    for ky in range(-Len_k,Len_k+1):
        #%run BandStructureFunction.ipynb
        #Energies[kx+Len_k]=np.real(E)
        kVect[0,kx+Len_k,ky+Len_k]=kx*a
        kVect[1,kx+Len_k,ky+Len_k]=ky*a
   

In [11]:
kVect

array([[[-9., -9., -9., -9., -9., -9., -9.],
        [-6., -6., -6., -6., -6., -6., -6.],
        [-3., -3., -3., -3., -3., -3., -3.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 9.,  9.,  9.,  9.,  9.,  9.,  9.]],

       [[-9., -6., -3.,  0.,  3.,  6.,  9.],
        [-9., -6., -3.,  0.,  3.,  6.,  9.],
        [-9., -6., -3.,  0.,  3.,  6.,  9.],
        [-9., -6., -3.,  0.,  3.,  6.,  9.],
        [-9., -6., -3.,  0.,  3.,  6.,  9.],
        [-9., -6., -3.,  0.,  3.,  6.,  9.],
        [-9., -6., -3.,  0.,  3.,  6.,  9.]]])